In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests
import asyncio
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds

In [20]:
import json
import requests
import time
from google.oauth2 import service_account
from googleapiclient.discovery import build

# # 서비스 계정 키 파일 경로
# SERVICE_ACCOUNT_FILE = 'path_to_service_account.json'

# 스프레드시트 ID
SPREADSHEET_ID = '1n-ihizmp-hF_hbeyi91n4CyXmvnX7Q23JGXGCwU1yQo'

# API 인증 및 서비스 생성
# credentials = service_account.Credentials.from_service_account_file(
#     SERVICE_ACCOUNT_FILE,
#     scopes=['https://www.googleapis.com/auth/spreadsheets']
# )

credentials = oauthByWeb();

service = build('sheets', 'v4', credentials=credentials)

# 트래커 API 설정
API_URL = 'https://apis.tracker.delivery/graphql'
CLIENT_ID = '1uja2e6bh8q4s7qf272ae88s5s'
CLIENT_SECRET = 'dgce4ft20dogp8i05dtqi2di5dvl0tkjvlnn5nne7fik225tkao'

def fetch_carrier_id(search_text):
    query = '''query CarrierList($searchText: String!, $after: String) {
        carriers(searchText: $searchText, first: 10, after: $after) {
            pageInfo {
                hasNextPage
                endCursor
            }
            edges {
                node {
                    id
                    name
                }
            }
        }
    }'''

    payload = {
        'query': query,
        'variables': {
            'searchText': search_text,
            'after': None
        }
    }

    headers = {
        'Authorization': f'TRACKQL-API-KEY {CLIENT_ID}:{CLIENT_SECRET}',
        'Content-Type': 'application/json'
    }

    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code == 200:
        json_response = response.json()
        print("json_response")
        print(json_response)
        if json_response.get('errors'):
            return None

        if json_response.get('data'):
            if json_response['data']['carriers']['edges']:
                return json_response['data']['carriers']['edges'][0]['node']['id']
    return None

def track_package(data):
    rows_to_update = []
    for i, row in enumerate(data):
        if len(row) < 13:
            continue

        if not row[13] or not row[13].isdigit():
            continue

        tracking_number = row[13]
        if not tracking_number or not tracking_number.isdigit():
            continue

        carrier_name = row[12]
        carrier_id = fetch_carrier_id(carrier_name)
        if not carrier_id:
            continue

        query = '''query Track($carrierId: ID!, $trackingNumber: String!) {
            track(carrierId: $carrierId, trackingNumber: $trackingNumber) {
                lastEvent {
                    time
                    status {
                        code
                        name
                    }
                    description
                }
            }
        }'''

        payload = {
            'query': query,
            'variables': {
                'carrierId': carrier_id,
                'trackingNumber': tracking_number
            }
        }

        headers = {
            'Authorization': f'TRACKQL-API-KEY {CLIENT_ID}:{CLIENT_SECRET}',
            'Content-Type': 'application/json'
        }

        response = requests.post(API_URL, headers=headers, json=payload)
        status_description = '미출고'
        if response.status_code == 200:
            json_response = response.json()
            if 'errors' in json_response:
                status_description = '미출고'
            elif 'data' in json_response and 'track' in json_response['data'] and 'lastEvent' in json_response['data']['track']:
                last_event = json_response['data']['track']['lastEvent']
                status_description = last_event['status']['name']

        row[15] = status_description
        row.extend([status_description])
        rows_to_update.append(row)

    return rows_to_update

def batch_update(sheet_id, rows_to_update, start_row):
    requests = []
    for i, row in enumerate(rows_to_update):
        requests.append({
            'updateCells': {
                'rows': [{
                    'values': [{'userEnteredValue': {'stringValue': str(cell)}} for cell in row]
                }],
                'fields': 'userEnteredValue',
                'start': {
                    'sheetId': sheet_id,
                    'rowIndex': start_row + i,
                    'columnIndex': 0
                }
            }
        })
    batch_update_request = {'requests': requests}
    service.spreadsheets().batchUpdate(
        spreadsheetId=SPREADSHEET_ID,
        body=batch_update_request
    ).execute()

def main(start_row, end_row):
    # 스프레드시트 데이터 가져오기
     # 데이터 가져올 범위 설정 (2059행부터 1500행까지)
    # start_row = 1800
    # num_rows = 1900
    # end_row = start_row + num_rows - 1
    sheet_name = 'test파이썬'
    range_name = f"\'test파이썬\'!A{start_row}:P{end_row}"
    
    sheet = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID, range=range_name).execute()
    data = sheet.get('values', [])
    sheets_metadata = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
    sheet_id = None
    for sheet in sheets_metadata['sheets']:
        if sheet['properties']['title'] == sheet_name:
            sheet_id = sheet['properties']['sheetId']
            break

    if sheet_id is None:
        print("Sheet '요것만!' not found")
        return

    # 데이터 처리
    rows_to_update = track_package(data[1:])  # 첫 번째 행은 헤더이므로 제외
    batch_update(sheet_id, rows_to_update, 1)  # 두 번째 행부터 업데이트

if __name__ == '__main__':
    a = [460, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900,2000, 2100]
    for i in range(1, len(a)):
        start_row = a[i - 1]
        end_row = a[i]
        main(start_row, end_row)


json_response
{'data': {'carriers': {'pageInfo': {'hasNextPage': False, 'endCursor': 'eyJpbmRleCI6OX0='}, 'edges': [{'node': {'id': 'kr.lotte', 'name': 'Lotte'}}, {'node': {'id': 'kr.lotte.global', 'name': 'Lotte Global'}}]}}}
json_response
{'data': {'carriers': {'pageInfo': {'hasNextPage': False, 'endCursor': 'eyJpbmRleCI6OX0='}, 'edges': [{'node': {'id': 'kr.lotte', 'name': 'Lotte'}}, {'node': {'id': 'kr.lotte.global', 'name': 'Lotte Global'}}]}}}
json_response
{'data': {'carriers': {'pageInfo': {'hasNextPage': False, 'endCursor': 'eyJpbmRleCI6OX0='}, 'edges': [{'node': {'id': 'kr.lotte', 'name': 'Lotte'}}, {'node': {'id': 'kr.lotte.global', 'name': 'Lotte Global'}}]}}}
json_response
{'data': {'carriers': {'pageInfo': {'hasNextPage': False, 'endCursor': 'eyJpbmRleCI6OX0='}, 'edges': [{'node': {'id': 'kr.lotte', 'name': 'Lotte'}}, {'node': {'id': 'kr.lotte.global', 'name': 'Lotte Global'}}]}}}
json_response
{'data': {'carriers': {'pageInfo': {'hasNextPage': False, 'endCursor': 'eyJpbm

IndexError: list index out of range